## POS tagging using modified Viterbi

### Importing Libraries

In [1]:
#Importing libraries
import nltk
import numpy as np
import pandas as pd
import time
import re
from sklearn.model_selection import train_test_split
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

##### Splitting the data into Train and Validation set   #test_set = validation

In [3]:
train, validation = train_test_split(nltk_data, train_size=0.95, test_size=0.05, random_state=101)

##### Pre-processing on Train set 

In [4]:
train_tagged_words = [tup for sent in train for tup in sent]  # gets all WORD + TAGS
tags = {tag for word, tag in train_tagged_words}    # create a Set of TAGS   
train_vocab = {word for word,tag in train_tagged_words}  # create a SET of Words in TRAIN

## Build the vanilla Viterbi based POS tagger

### Emission and Transition Probability 

In [5]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag=train_tagged_words):
    """
    Emission probability of word(w) and tag(t) is,
    Number of times word (w) has been tagged with (t) / numb of times t Appear

    :param word: the Word (W)
    :param tag: The  Tag (t) for word (W)
    :param train_bag: corpus of word and their tags
    :return: freq of (w|t), freg of tag (t)
    """
    tag_list = [pair for pair in train_bag if pair[1] == tag]
    count_tag = len(tag_list)  # total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0] == word]
    # now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
    return count_w_given_tag, count_tag


# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag=train_tagged_words):
    """
    Prob of trainsition of tag(t1) to tag(t2) = no of times (t2|t1)/ num of (t1) appears
    :param t2: tag
    :param t1: tag
    :param train_bag: Reference corpus
    :return: count of (t2|t1), count of t1
    """
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return count_t2_t1, count_t1


### Viterbi Algorithm 

In [6]:
# Viterbi Algorithm
def Viterbi(words, train_bag=train_tagged_words):
    """
    words:     Individual Words from Test data set for POS Tagging
    train_bag: WORD + TAGS of Train Data -->[('Reliance', 'NOUN'),('confirmed', 'VERB'),('the', 'DET'),('filing', 'NOUN')]
    """
    state = []
    T = list(set([pair[1] for pair in train_bag]))  # gives list of unique Tags
    for key, word in enumerate(words):
        # initialise list of probability column for a given observation
        
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]      # getting transition prob from Pandas DF created
            else:
                transition_p = tags_df.loc[state[-1], tag]

            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0] / word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))



#### Transition Matrix Creation (T * T)

    creating t * t transition matrix of tags, t= no of tags
    Matrix(i, j) represents P(jth tag after the ith tag)
    
    * Converting this Matrix to Dataframe for Better visulaization


In [7]:
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
        
        
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
del tags_matrix

In [8]:
tags_df.head()

,DET,CONJ,NUM,.,ADV,VERB,ADJ,ADP,PRON,NOUN,X,PRT
DET,0.005676,0.000483,0.022220,0.017993,0.012438,0.039850,0.204323,0.009540,0.003744,0.638087,0.045405,0.000242
CONJ,0.121339,0.000465,0.039981,0.034868,0.055323,0.156671,0.118085,0.052534,0.058113,0.349140,0.008833,0.004649
NUM,0.003276,0.013699,0.184932,0.117332,0.002978,0.018761,0.034247,0.036033,0.001489,0.350208,0.210542,0.026504
.,0.173335,0.057538,0.081003,0.093320,0.052324,0.089095,0.043963,0.091342,0.066349,0.222242,0.026971,0.002427
ADV,0.069891,0.006956,0.030474,0.137131,0.080490,0.343491,0.129182,0.118582,0.014906,0.031467,0.023186,0.014243


####  Pre-Processing on validation data set

In [9]:
validation_tagged_words = [tup for sent in validation for tup in sent]  # list of word and tag
print('validation_tagged_words :', validation_tagged_words[:5])
validation_vocab = [tup[0] for sent in validation for tup in sent]  # only WORDS
print('validation_vocab :',validation_vocab[:5])

validation_tagged_words : [('The', 'DET'), ('company', 'NOUN'), ('said', 'VERB'), ('0', 'X'), ('it', 'PRON')]
validation_vocab : ['The', 'company', 'said', '0', 'it']


#### Running Vanilla Viterbi on Validation Vocablary Data

In [10]:
start = time.time()
tagged_seq_vanilla = Viterbi(validation_vocab)  # Viterbi on sample Test Data
end = time.time()
difference = end-start


#### Evaluating tagging accuracy for Vanilla Viterbi Algorithm

In [11]:
correct_tags_vanilla = [i for i, j in zip(tagged_seq_vanilla, validation_tagged_words) if i == j]
accuracy_vanilla = len(correct_tags_vanilla)/len(tagged_seq_vanilla)

print("Time taken in Minutes: ", difference/60)
print('Total Correct Tags :', len(correct_tags_vanilla))
print('Vanilla Viterbi Algorithm Accuracy: ',accuracy_vanilla*100)

Time taken in Minutes:  32.95615941286087
Total Correct Tags : 4663
Vanilla Viterbi Algorithm Accuracy:  90.91440826671867


## Solve the problem of unknown words

UNKNOWN WORDS : it is defined as words that are there in TEST data, but not in TRAIN data set.
                
                subtraction of set is used to get Unknown words

In [12]:
unknown_vocab = list(set(validation_vocab) - set(train_vocab))
print(len(unknown_vocab))

308


    So there are 308, words which are not present in the training vocablary set

### Finding Most Frequent Tag for unknown Words


In [13]:
unknown_word_tag_freq = dict([(key, 0) for key in tags])   # Dictionary Creation

for ele in validation_tagged_words:
    if ele[0] in unknown_vocab:
        unknown_word_tag_freq[ele[1]] = unknown_word_tag_freq.get(ele[1],0)+1

print(unknown_word_tag_freq)

{'DET': 0, 'CONJ': 0, 'NUM': 29, '.': 0, 'ADV': 2, 'VERB': 62, 'ADJ': 44, 'ADP': 0, 'PRON': 0, 'NOUN': 167, 'X': 14, 'PRT': 0}


    Note :  From above we see that Most unknown words in the vocablary belongs to the Tag "NOUN" 
            and Assuming a Few Numerical Format Data belong to "NUM".

### Modified Viterbi Algorithm - 1
    Considering the Observation from Above we will try to Improve the Vanilla Viterbi Algorithm using 
    morphological cues.
    1. Identify Numbers using re.search and tag them as NUM
    2. Assign all Unknown words the most Frequent Tags ie NOUN in our case

In [14]:
# Viterbi Heuristic
def Viterbi_Rule_Based(words, unknown_word_set, train_bag=train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        # Using a few Rules to Identify  Numbers 
        if re.search(r'^-?[0-9]+(.[0-9]+)?$', word):
            if word == '0':
                state_max = 'X'
            else:
                state_max = 'NUM'
        elif re.search(r'\*[A-Za-z]*\*-[0-9]*', word):
            state_max = 'X'
        elif re.search(r'-[A-Za-z]*-', word):
            state_max = '.'
        elif word in unknown_word_set:   # tagging unknown words to the Most occured Tag 
            state_max = 'NOUN'
            
        # initialise list of probability column for a given observation
        else:
            p = []
            for tag in T:
                if key == 0:
                    transition_p = tags_df.loc['.', tag]
                else:
                    transition_p = tags_df.loc[state[-1], tag]

                # compute emission and state probabilities
                emission_p = word_given_tag(words[key], tag)[0] / word_given_tag(words[key], tag)[1]
                state_probability = emission_p * transition_p
                p.append(state_probability)

            pmax = max(p)
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))


In [15]:
start = time.time()
tagged_seq_rule = Viterbi_Rule_Based(validation_vocab, unknown_vocab)  # Viterbi on sample Test Data
end = time.time()
difference = end-start


#### Evaluating tagging accuracy for Viterbi Algorithm - 1

In [16]:
correct_tags_rule = [i for i, j in zip(tagged_seq_rule, validation_tagged_words) if i == j]
accuracy_rule_based = len(correct_tags_rule)/len(tagged_seq_rule)

print("Time taken in Minutes: ", difference/60)
print('Total Correct Tags :', len(correct_tags_rule))
print('Rule based Viterbi Algorithm Accuracy: ',accuracy_rule_based*100)

Time taken in Minutes:  8.038105670611063
Total Correct Tags : 4872
Rule based Viterbi Algorithm Accuracy:  94.98927666211738


### Modified Viterbi Algorithm - 2
    
    In this Modified version we try to incorporate the probabilistic approch to unknown vacablary set
    1. We use Rule based Approch to Tag NUM and X
    2. We use a probabilistic Rule that: 
        for words not in the Training Corpus, their State Probability = Transition Probability

In [17]:

# Viterbi algiorithm with Rule based +  Probabilistic approch
def Viterbi_rule_prob(words, unknown_list, train_bag=train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))

    for key, word in enumerate(words):
        #print(key, word)
        if re.search(r'^-?[0-9]+(.[0-9]+)?$', word):
            if word == '0':
                state_max = 'X'
            else:
                state_max = 'NUM'
        elif re.search('ing',word) or re.search('ed', word):
            state_max = 'VERB'
        elif re.search(r'\*[A-Za-z]*\*-[0-9]*', word):
            state_max = 'X'
        elif re.search(r'-[A-Za-z]*-', word):
            state_max = '.'
        elif word in unknown_list:
            #print(word)
            p=[]
            for tag in T:
                transition_p = tags_df.loc[state[-1], tag]
                state_probability = transition_p
                p.append(state_probability)
            pmax = max(p)
            state_max = T[p.index(pmax)]
        # initialise list of probability column for a given observation
        else:
            p = []
            for tag in T:
                if key == 0:
                    transition_p = tags_df.loc['.', tag]
                else:
                    transition_p = tags_df.loc[state[-1], tag]

                # compute emission and state probabilities
                emission_p = word_given_tag(words[key], tag)[0] / word_given_tag(words[key], tag)[1]
                state_probability = emission_p * transition_p
                p.append(state_probability)

            pmax = max(p)
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))


In [18]:
start = time.time()
tagged_seq_prob = Viterbi_rule_prob(validation_vocab, unknown_vocab)  # Viterbi on sample Test Data
end = time.time()
difference = end-start

#### Evaluating tagging accuracy for Viterbi Algorithm - 2 

In [19]:
correct_tags_prob = [i for i, j in zip(tagged_seq_prob, validation_tagged_words) if i == j]
accuracy_probabilistic = len(correct_tags_prob)/len(tagged_seq_prob)

print("Time taken in Minutes: ", difference/60)
print('Total Correct Tags :', len(correct_tags_prob))
print('Probality Viterbi Algorithm Accuracy: ',accuracy_probabilistic*100)

Time taken in Minutes:  7.954983973503113
Total Correct Tags : 4803
Probality Viterbi Algorithm Accuracy:  93.64398518229675


### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [21]:
print('The Vanilla Viterbi\n')
print('Accuracy of Vanilla Viterbi Algorithm : ',accuracy_vanilla*100)

The Vanilla Viterbi

Accuracy of Vanilla Viterbi Algorithm :  90.91440826671867


In [22]:
print('Modified Viterbi Algorithm -2 \n uses a Probabilistic rule for unknown words that State Probality of Unknown words  =  Transition Probablity\n')
print('Accuracy of Probality based Viterbi Algorithm : ',accuracy_probabilistic*100)

Modified Viterbi Algorithm -2 
 uses a Probabilistic rule for unknown words that State Probality of Unknown words  =  Transition Probablity

Accuracy of Probality based Viterbi Algorithm :  93.64398518229675


In [23]:
print('Modified Viterbi Algorithm - 1 \n uses a Rule based approch for unknown words ie. tagging unknown words with Most occured POS Tag\n')
print('Accuracy of Probality based Viterbi Algorithm : ',accuracy_rule_based*100)

Modified Viterbi Algorithm - 1 
 uses a Rule based approch for unknown words ie. tagging unknown words with Most occured POS Tag

Accuracy of Probality based Viterbi Algorithm :  94.98927666211738


### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [25]:
# incorrect tagging by Vanilla Viterbi
incorrect_tags_vanilla = [j for i, j in zip(tagged_seq_vanilla, validation_tagged_words) if i != j]


In [27]:
corrected_by_Algo_rule = []
for ele in incorrect_tags_vanilla:
    if ele in correct_tags_rule:
        corrected_by_Algo_rule.append(ele[0])
        

In [28]:
print('The Following Tags got corrected in the Rule based modified Viterbi Algorithm\n')
print(corrected_by_Algo_rule)

The Following Tags got corrected in the Rule based modified Viterbi Algorithm

['Deere', 'wave', 'Eveready', 'nickname', 'vagabond', 'existence', 'Miklos', 'Nemeth', '2.6', 'indictment', 'Vice', 'Sherwin', 'Carbide', 'that', '737.5', '3.01', 'ethanol', 'fleet', 'Mount', 'Clemens', 'NYSE', 'Symbol', 'CSV', 'Ketchum', 'Pittsburgh', 'Braun', 'investor-relations', 'marketing-communications', 'escort', 'busloads', 'wives', 'Speedway', 'traffic', 'flood', 'that', '*T*-211', 'rounds', 'verge', 'memo', '89,500', 'that', 'fashion', 'sky', 'railcar', 'platforms', 'Trailer', 'Train', 'up', 'resumption', 'shuttle', 'launch-vehicle', 'engines', '12.82', '133.7', '94', '23.25', 'TREASURY', 'BILLS', 'Results', '7.78', '7.62', 'crapshoot', 'that', '*T*-126', 'document', 'known', 'Form', '8300', 'self', 'sufficiency', 'Payouts', 'Excision', 'riders', 'that', 'prerogative', 'Demand', '*T*-236', 'inside', 'Spending', '2.6', '99.1', 'observance', 'Saints', "'", 'Day', 'Datapoint', 'variation', '240,000', 

In [29]:
corrected_by_Algo_prob = []
for ele in incorrect_tags_vanilla:
    if ele in correct_tags_prob:
        corrected_by_Algo_prob.append(ele[0])

In [30]:
print('The Following Tags got corrected in the Rule based modified Viterbi Algorithm\n')
print(corrected_by_Algo_prob)

The Following Tags got corrected in the Rule based modified Viterbi Algorithm

['Deere', 'traced', 'wave', 'nickname', 'glamorize', 'authorized', 'Miklos', 'Nemeth', '2.6', 'indictment', 'Sherwin', 'manipulate', 'Carbide', 'that', '737.5', '3.01', 'centralized', 'ethanol', 'fleet', 'Clemens', 'NYSE', 'Symbol', 'CSV', 'Ketchum', 'Pittsburgh', 'investor-relations', 'escort', 'busloads', 'raced', 'labeling', 'reopened', 'flood', 'that', '*T*-211', 'knocked', '*-128', '*-128', 'synchronized', 'rounds', 'descending', 'verge', 'disseminate', 'memo', '89,500', 'that', 'Bucking', 'drew', 'sky', 'railcar', 'platforms', 'up', 'resumption', 'shuttle', 'launch-vehicle', 'engines', '12.82', '133.7', '94', 'delisted', '23.25', 'TREASURY', 'BILLS', 'Results', '7.78', '7.62', 'crapshoot', 'that', '*T*-126', 'receives', 'document', 'known', '8300', 'self', 'sufficiency', 'Payouts', 'Excision', 'riders', 'that', 'trespass', 'prerogative', 'Demand', '*T*-236', 'inside', '2.6', '99.1', 'muted', 'observanc

## Conclusions

In our Two Different Approches taken ie:

    1. Rule Based : where Numbers appering were tagged as "NUM" and 
                    All words not in the Training corpus was tagged as "NOUN",
                    as it was the most appearing TAG
    
    2. Probablistic : Here numbers were tagged as "NUM" --Rule and
                      For all Words of Test set not in Training Set, 
                      their State_probability was set as Transision Prob.
 
#### Result :
    
    * The Vanila Viterbi had an Accuracy of approx 90%
    * Probablistic approch improved this to approx 93%
    * Finally we observe that our Rule based Modification gave an Accuracy  approx 95%
    